In [1]:
import pandas as pd
from pymongo import MongoClient
import random

from mongo_aggregation_verbs import *

mongo_client = MongoClient('18.236.138.158', 27016)
database_reference = mongo_client.twitter

In [2]:
#Use the bad hash tag to filter out the bad tweets
database_reference.collection_names()

collection_reference = database_reference.instructor_test_group

test_group = database_reference.instructor_test_group

In [3]:
match_empty_url_arrays = { MATCH : { "entities.urls" : [] } }

list(test_group.aggregate(
    [
        match_empty_url_arrays,
        { COUNT : "text" }
    ]
))

[{'text': 11121}]

In [4]:
#filtering out bad hashtag. 
job_hashtags = ['job', 'jobs', 'hiring', 'careerarc']
location_hashtags = ['california', 'losangeles', 'la', 'santamonica', 'glendale', 'paloalto']
match_not_in_bad = { MATCH : { "text" : { "$in" : job_hashtags + location_hashtags } } }
project_to_text_keep_id = { PROJECT : { "text" : "$entities.hashtags.text" } }
project_to_id = { PROJECT : { "_id" : 1 } }

bad_ids = list(test_group.aggregate(
    [
        match_non_empty_hashtag_arrays,
        project_to_text_keep_id,
        unwind_text,
        project_to_lower,
        match_not_in_bad,
        project_to_id
    ]
))
bad_ids[:10], len(bad_ids)

([{'_id': ObjectId('5a73683636dd5f00016c7fad')},
  {'_id': ObjectId('5a73683636dd5f00016c7fad')},
  {'_id': ObjectId('5a73683636dd5f00016c7fad')},
  {'_id': ObjectId('5a73683636dd5f00016c7fad')},
  {'_id': ObjectId('5a73683636dd5f00016c7fad')},
  {'_id': ObjectId('5a6da1bd36dd5f0001690696')},
  {'_id': ObjectId('5a6da1bd36dd5f0001690696')},
  {'_id': ObjectId('5a6da1bd36dd5f0001690696')},
  {'_id': ObjectId('5a6df39136dd5f0001691533')},
  {'_id': ObjectId('5a6df39136dd5f0001691533')}],
 1835)

In [5]:
#this id have the bad tweets that you dont want. 
bad_ids = [bad_id['_id'] for bad_id in bad_ids]
bad_ids[:10]

[ObjectId('5a73683636dd5f00016c7fad'),
 ObjectId('5a73683636dd5f00016c7fad'),
 ObjectId('5a73683636dd5f00016c7fad'),
 ObjectId('5a73683636dd5f00016c7fad'),
 ObjectId('5a73683636dd5f00016c7fad'),
 ObjectId('5a6da1bd36dd5f0001690696'),
 ObjectId('5a6da1bd36dd5f0001690696'),
 ObjectId('5a6da1bd36dd5f0001690696'),
 ObjectId('5a6df39136dd5f0001691533'),
 ObjectId('5a6df39136dd5f0001691533')]

In [6]:
not_in_bad_ids = { "$nin" : bad_ids }

In [7]:
not_in_bad_ids_and_no_url = { 
    "_id"           : not_in_bad_ids, 
    "entities.urls" : []
}

just_the_text = {
    "text" : 1,
    "_id"  : 0
}

In [8]:
test_group.find_one(
    not_in_bad_ids_and_no_url,
    just_the_text
)

{'text': "@SincerelyLegit Lol why at night? If it's boring I'll fall asleep"}

In [9]:
cur  = test_group.find(
    not_in_bad_ids_and_no_url,
    just_the_text
)

tweets = list(cur)
tweet_text = pd.DataFrame(tweets)

In [10]:
len(tweet_text)

11102

In [11]:
tweet_text.head()

,text
0,@SincerelyLegit Lol why at night? If it's bori...
1,@godtributes Lol
2,@KingBeyonceStan I need to binge on both now b...
3,@_QUEENSharnay Naw I’d i can’t put this skip o...
4,@sannicolaso I have things to do at home but y...


In [12]:
#Removing urls from the tweets. 
tweet_text.text = tweet_text.text.str.replace('http\S+|www.\S+', '', case=False)

In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer #get dummies. Example: the brown fox, the red dog, the blue fox
#blue brown dog fox red the
# 0 1 0 1 0 1
# 0 0 1 0 1 1
# 1 0 0 1 0 1

In [14]:
tfidf = TfidfVectorizer(stop_words='english')
tfidf.fit(tweet_text.text)
word_occurence = tfidf.transform(tweet_text.text).todense() #sparsing. 

In [15]:
words = tfidf.get_feature_names()
word_sample = random.sample(words, 20)
word_occurence_m = pd.DataFrame(word_occurence, columns=words)
word_occurence_m[word_sample].head()

,electronic,danbury,dijon,masih,logic301,wouldnt,shows,saidnooneever,arrow,freakin,shanedawson,49da,2009superglide,345,billy,yunggoth300,ac0ord,hoopsnbrews,malaa,subtly
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
from sklearn.decomposition import LatentDirichletAllocation #PCA 

In [17]:
lda = LatentDirichletAllocation(n_topics=10, learning_method='batch')
lda.fit(word_occurence)

/opt/conda/lib/python3.6/site-packages/sklearn/decomposition/online_lda.py:294: DeprecationWarning: n_topics has been renamed to n_components in version 0.19 and will be removed in 0.21
  DeprecationWarning)


LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=10, mean_change_tol=0.001,
             n_components=10, n_jobs=1, n_topics=10, perp_tol=0.1,
             random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [18]:
lda_df = pd.DataFrame(lda.components_, columns=words).T

In [19]:
def filter_topic(lda_df, index, threshold):
    return (lda_df[lda_df[index] > threshold][index]
            .sort_values(ascending=False))

In [20]:
filter_topic(lda_df, 0, 10) # eigenvectors

just       20.819628
like       20.394539
good       14.801889
lmao       13.103616
lol        12.180283
new        11.680016
friends    10.724212
need       10.182631
Name: 0, dtype: float64

In [21]:
filter_topic(lda_df, 1, 10)

love     20.838463
lol      18.938164
just     16.481933
money    14.309892
like     13.157434
god      11.885029
good     11.602704
miss     11.080954
Name: 1, dtype: float64

In [22]:
filter_topic(lda_df, 2, 10)

like    18.124962
just    14.792892
don     14.724669
yes     13.083611
need    11.236664
know    10.219134
love    10.214623
Name: 2, dtype: float64

In [23]:
filter_topic(lda_df, 3, 10) #(3,5) to get more results. 

wanna    17.119917
fuck     16.231703
im       15.436408
sleep    14.864686
ain      12.064060
sick     11.940254
la       11.224147
just     11.136537
night    10.928656
good     10.844175
wow      10.512912
don      10.117338
gonna    10.032202
Name: 3, dtype: float64

In [24]:
filter_topic(lda_df, 4, 10)

realdonaldtrump    22.758919
love               11.772810
morning            11.254855
best               10.553939
Name: 4, dtype: float64

In [25]:
filter_topic(lda_df, 5, 10)

mood    18.262415
just    13.850486
love    11.247333
lol     10.727259
Name: 5, dtype: float64

In [26]:
filter_topic(lda_df, 6, 6)

got      15.866185
know     14.637163
like      9.398057
just      9.212843
hell      9.002791
oh        8.527935
work      7.605461
day       7.103946
love      6.860305
lol       6.768655
don       6.626128
going     6.301424
Name: 6, dtype: float64

In [27]:
filter_topic(lda_df, 7, 7)

like     10.581121
just      8.522505
great     7.884895
amp       7.306289
funny     7.069859
Name: 7, dtype: float64

In [28]:
filter_topic(lda_df, 8, 7)

want        11.411624
happy        9.793577
love         8.602508
know         8.300156
just         8.081254
dude         7.817839
birthday     7.545315
people       7.332833
Name: 8, dtype: float64

In [29]:
filter_topic(lda_df, 9, 7)

time     13.020058
bitch     9.720293
make      9.285208
just      8.828299
need      8.603122
black     7.847273
old       7.829164
shit      7.780429
life      7.693450
love      7.577069
lol       7.041919
Name: 9, dtype: float64